Log in to facebook?



In [3]:

var enterFacebook = () => {
    console.log('Facebook: Sign in required');
    var credentials = getCredentials('facebook.com');
    return client.click('input[name*="email"]')
    .keys(credentials.email)
    .pause(1000)
    .then(() => console.log('Facebook: Require password'))
    .click('input[name*="pass"]')
    .keys(credentials.pass)
    .submitForm('[type="submit"]')
    .pause(2000)
    .isExisting('.cp-challenge-form')
    .then(is => {
        if(is) {
            throw new Error('captcha');
        }
    });
}

var loginFacebook = () => {
    return client
    .getUrl().then(url => {
        var loggedIn = url.indexOf('facebook') > -1 && url.indexOf('login') == -1 && url !== 'https://www.facebook.com'
        return loggedIn
            ? client
            : client.url('https://www.facebook.com/')
    });
}
module.exports = loginFacebook;
loginFacebook;



[Function: loginFacebook]

In [4]:
var importer = require('../Core');

$$.async()
var runSeleniumCell;
importer.interpretAll(['selenium cell'])
    .then(r => {
        runSeleniumCell = r[0].runInNewContext();
        return runSeleniumCell('log in facebook');
    })
    .then(loginFacebook => loginFacebook())
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));



Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Initializing webdriver on localhost
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]


{ state: 'success',
  sessionId: '345cb77a-417f-4e49-b4f3-4e3cd526b7f7',
  hCode: 976732172,
  value: null,
  class: 'org.openqa.selenium.remote.Response',
  status: 0 }